# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770816926974                   -0.53    9.0         
  2   -2.772143921504       -2.88       -1.30    1.0    204ms
  3   -2.772170153216       -4.58       -2.62    1.0    161ms
  4   -2.772170720316       -6.25       -3.92    2.0    190ms
  5   -2.772170722732       -8.62       -4.31    2.0    231ms
  6   -2.772170723007       -9.56       -5.40    1.0    167ms
  7   -2.772170723015      -11.12       -5.96    2.0    196ms
  8   -2.772170723015      -12.99       -6.56    1.0    200ms
  9   -2.772170723015      -13.86       -7.40    2.0    211ms
 10   -2.772170723015   +  -14.75       -8.10    2.0    226ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770753399937                   -0.53    8.0         
  2   -2.772050668940       -2.89       -1.31    1.0    159ms
  3   -2.7

1.773558063493046

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770751899086                   -0.53    9.0         
  2   -2.772052232574       -2.89       -1.30    1.0    220ms
  3   -2.772082863402       -4.51       -2.67    1.0    162ms
  4   -2.772083416719       -6.26       -4.05    2.0    227ms
  5   -2.772083417781       -8.97       -5.13    1.0    168ms
  6   -2.772083417810      -10.54       -5.70    2.0    205ms
  7   -2.772083417811      -12.34       -6.24    1.0    171ms
  8   -2.772083417811      -13.44       -7.38    1.0    198ms
  9   -2.772083417811   +  -15.05       -7.46    2.0    219ms
 10   -2.772083417811   +  -14.57       -8.93    1.0    213ms

Polarizability via ForwardDiff:       1.772534968771911
Polarizability via finite difference: 1.773558063493046
